In [1]:
import torch
from PIL import Image
from torchvision import transforms, models
from ModelAnalysis import ModelStatsAggregator, ModelStatAnalyser
from sys import version
import timm
from tqdm import tqdm
import urllib
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
version

'3.9.5 (default, Aug 29 2021, 19:01:31) \n[GCC 9.3.0]'

In [2]:
model_dict = {
    'ssd': torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd'),
    'lenet': torch.hub.load('pytorch/vision:v0.10.0', 'googlenet'),
    'yolov5s': torch.hub.load('ultralytics/yolov5', 'yolov5s'),
    'yolov5m': torch.hub.load('ultralytics/yolov5', 'yolov5m'),
    'yolov5l': torch.hub.load('ultralytics/yolov5', 'yolov5l'),
    'yolov5x': torch.hub.load('ultralytics/yolov5', 'yolov5x'),
    'alexnet': models.alexnet(),
    'squeezenet_1_0': models.squeezenet1_1(),
    'squeezenet_1_1': models.squeezenet1_0(),
    'googlenet': models.googlenet(),
    'shufflenet_v2_x0_5': models.shufflenet_v2_x0_5(),
    'shufflenet_v2_x1_0': models.shufflenet_v2_x1_0(),
}

Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
Using cache found in /home/sultan/.cache/torch/hub/pytorch_vision_v0.10.0
/home/sultan/CNN_Network_Analysis/.venv/lib/python3.9/site-packages/torchvision/models/googlenet.py:77: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (GeForce RTX 2070 SUPER, 7979.1875MB)

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /home/sultan/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-29 torch 1.9.0+cu102 CUDA:0 (G

In [3]:
# prepare sample inputs
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg'
]
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

inputs = [utils.prepare_input(uri) for uri in uris]
ssd_input_batch = utils.prepare_tensor(inputs)

Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [4]:
stats_dict = ModelStatAnalyser.get_models_stats_dict(model_dict, input_batch, ssd_input_batch) 

In [5]:
model_batch_size = 4
model_list = timm.list_models(exclude_filters=['levit_*'], pretrained=False)
in_shapes = []
for model_batch_idx in tqdm(range(0, len(model_list), model_batch_size)):
    for model_name in model_list[model_batch_idx: model_batch_idx+model_batch_size]:
        model = timm.create_model(model_name, pretrained=False)
        config = resolve_data_config({}, model=model)
        transform = create_transform(**config)

        url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
        urllib.request.urlretrieve(url, filename)
        img = Image.open(filename).convert('RGB')
        tensor = transform(img).unsqueeze(0) # transform and add batch dimension
        in_tensor_shape = tensor.size()
        if in_tensor_shape[2] == 224 and in_tensor_shape[3] == 224:       
            in_shapes.append(in_tensor_shape)
            model_dict = {}
            model_dict[model_name] = model
            try:
                res_stats_dict = ModelStatAnalyser.get_models_stats_dict(model_dict, tensor, ssd_input_batch)
            except Exception as e:
                print(f"Model name {model_name}")
                raise e
            stats_dict = stats_dict | res_stats_dict

  0%|          | 0/152 [00:00<?, ?it/s]/home/sultan/CNN_Network_Analysis/.venv/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
100%|██████████| 152/152 [07:36<00:00,  3.00s/it]


In [6]:
total_networks = len(stats_dict.keys())
networks_with_kernel = {f'{k}': 0 for k in range(1, 12)}
for network, stats in stats_dict.items():
    for kernel_size in networks_with_kernel.keys():
        if kernel_size in stats['kernel']._dict:
            networks_with_kernel[kernel_size] += 1
print(networks_with_kernel)

{'1': 336, '2': 13, '3': 356, '4': 19, '5': 66, '6': 0, '7': 156, '8': 11, '9': 8, '10': 0, '11': 3}


In [15]:
from numpy import linspace
from math import floor
longest_network_length = max([len(stats['raw_stats'].keys()) for model, stats in stats_dict.items()])
normalized_model_intermediate_layer_size = {}
for model, stats in stats_dict.items():
    normalized_model_intermediate_layer_size[model] = [0]*longest_network_length
    model_length = len(stats['intermediate_layer_sizes'])
    idx_list, step = linspace(0, longest_network_length, model_length, retstep=True)
    idx_list, step = [floor(idx) for idx in idx_list], floor(step)
    input_layers = stats['intermediate_layer_sizes'][0:-1]
    output_layers = stats['intermediate_layer_sizes'][1:]
    layer_sizes = zip(input_layers, output_layers)
    for idx in idx_list[:-1]:
        try:
            in_layer, out_layer = next(layer_sizes)
        except:
            print(idx)
            raise Exception('WOOPS!')
        try:
            in_layer, out_layer = in_layer[1][-1], out_layer[1][-1]
        except:
            print(in_layer)
            print(out_layer)
            raise Exception('WOOPS!')
        interpolated_layers = [floor(layer_size) for layer_size in linspace(in_layer, out_layer, step)]
        for interpolation_idx in range(idx, idx+step):
            try:
                normalized_model_intermediate_layer_size[model][interpolation_idx] = interpolated_layers[interpolation_idx-idx] 
            except:
                print(interpolation_idx)
                raise Exception('WOOPS')
    # interpolate ith linspace + linspace step size when length of networks < max length
        

In [17]:
normalized_model_intermediate_layer_size['alexnet']

[224,
 220,
 217,
 214,
 211,
 208,
 205,
 202,
 199,
 196,
 193,
 190,
 187,
 183,
 180,
 177,
 174,
 171,
 168,
 165,
 162,
 159,
 156,
 153,
 150,
 147,
 143,
 140,
 137,
 134,
 131,
 128,
 125,
 122,
 119,
 116,
 113,
 110,
 107,
 103,
 100,
 97,
 94,
 91,
 88,
 85,
 82,
 79,
 76,
 73,
 70,
 67,
 63,
 60,
 57,
 54,
 51,
 48,
 45,
 42,
 39,
 36,
 33,
 30,
 27,
 27,
 26,
 26,
 26,
 26,
 25,
 25,
 25,
 25,
 25,
 24,
 24,
 24,
 24,
 23,
 23,
 23,
 23,
 23,
 22,
 22,
 22,
 22,
 21,
 21,
 21,
 21,
 21,
 20,
 20,
 20,
 20,
 20,
 19,
 19,
 19,
 19,
 18,
 18,
 18,
 18,
 18,
 17,
 17,
 17,
 17,
 16,
 16,
 16,
 16,
 16,
 15,
 15,
 15,
 15,
 14,
 14,
 14,
 14,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,

In [7]:
ModelStatsAggregator.get_aggregate_kernel_stats_as_percentages(stats_dict)

{'7': 0.008996066358816487,
 '1': 0.6101932615016248,
 '3': 0.35016247648366683,
 '11': 0.0001026167265264238,
 '5': 0.020489139729775955,
 '16': 0.001573456473405165,
 '4': 0.0012998118693347016,
 '2': 0.003728407730460065,
 '32': 0.0003078501795792714,
 '9': 0.0024628014366341712,
 '14': 0.0001026167265264238,
 '8': 0.000581494783649735}

In [40]:
expected_kernel_sizes = list(range(1,12))
count_of_models_with_kernel_size = {f'{k}': 0 for k in expected_kernel_sizes}
for model, stats in stats_dict.items():
    for size in expected_kernel_sizes:
        count_of_models_with_kernel_size[f'{size}'] += (1 if stats['kernel'][f'{size}'] > 0 else 0)
print(count_of_models_with_kernel_size)

{'1': 380, '2': 22, '3': 359, '4': 22, '5': 66, '6': 46, '7': 156, '8': 11, '9': 8, '10': 0, '11': 0}


In [21]:
stats_dict['ssd']['intermediate_layer_sizes']

[(270000, [1, 3, 300, 300]),
 (360000, [1, 64, 75, 75]),
 (360000, [1, 64, 75, 75]),
 (360000, [1, 64, 75, 75]),
 (360000, [1, 64, 75, 75]),
 (1440000, [1, 256, 75, 75]),
 (360000, [1, 64, 75, 75]),
 (360000, [1, 64, 75, 75]),
 (1440000, [1, 256, 75, 75]),
 (360000, [1, 64, 75, 75]),
 (360000, [1, 64, 75, 75]),
 (1440000, [1, 256, 75, 75]),
 (720000, [1, 128, 75, 75]),
 (184832, [1, 128, 38, 38]),
 (1440000, [1, 256, 75, 75]),
 (739328, [1, 512, 38, 38]),
 (184832, [1, 128, 38, 38]),
 (184832, [1, 128, 38, 38]),
 (739328, [1, 512, 38, 38]),
 (184832, [1, 128, 38, 38]),
 (184832, [1, 128, 38, 38]),
 (739328, [1, 512, 38, 38]),
 (184832, [1, 128, 38, 38]),
 (184832, [1, 128, 38, 38]),
 (739328, [1, 512, 38, 38]),
 (369664, [1, 256, 38, 38]),
 (369664, [1, 256, 38, 38]),
 (739328, [1, 512, 38, 38]),
 (1478656, [1, 1024, 38, 38]),
 (369664, [1, 256, 38, 38]),
 (369664, [1, 256, 38, 38]),
 (1478656, [1, 1024, 38, 38]),
 (369664, [1, 256, 38, 38]),
 (369664, [1, 256, 38, 38]),
 (1478656, [1,